<h1>Web Scraping Amazon to get make a DF<h1>
<h3>  Objective:</h3> <p>From each amazon search generate a df on the results with their price, product name and product link.</p>
<bh>
<h3> Assuptions:</h3>
<p>For this particular case we will asume the folling <a href="https://www.familyhandyman.com/list/best-smart-dog-collars/">'article'</a>  is a formal study for the best Dog Smart Collar's in the market. So based on it we will:</p>
<br>
<p> 
-Read the articule using BeautifulSoup and Slenium libraries<br>
-Get the products from the market study<br>
-Using the global leader amazon ecommerce we will look up for them and gather info on al searches<br>
-Save into a DF for later analysis<br>

</p>


## Libraries:

In [1]:
# Beautiful Soup libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import smtplib #auto emailing 
import time
import datetime
from datetime import datetime
import pytz
import itertools
import csv
import re

In [2]:
# Selenium Libraries
import selenium
from xml.etree.ElementTree import QName
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException
import os


#Chrome WebDriver    
chrome_options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values.notifications' : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_path = r"/Users/carlosarreola/Data Science - Trainning/NLP Project Example/chromedriver 2"
driver = webdriver.Chrome(executable_path=chrome_path,options=chrome_options)
wait = WebDriverWait(driver, 10, poll_frequency=1, ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException])
download_wait = WebDriverWait(driver, 1000, poll_frequency=1, ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException])


## Functions

In [3]:
# EnterPage
def enterPage(site):
    driver.get(site)
    try:
        element_present = EC.presence_of_element_located((By.ID, 'a-page'))
        wait.until(element_present)
        status = "Site Loaded Correctly"
        return status      
    except Exception as e:
        status = "Error Site Not Loading, Error: " + str(e)
        return status


In [4]:
#Get soup from any url
def get_soup(url):
    # Web Connection
    link = url
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36", "Accept-Encoding":"gzip, deflate, br", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(link, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    return soup2
        

In [5]:
# Function to extract data from a specific product
def check_price(URL):   
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36", "Accept-Encoding":"gzip, deflate, br", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    prod = soup2.find(id='productTitle').get_text()
    price = soup2.find(id='apex_desktop').get_text()

    price = price.strip()[1:]
    price = price.split('\n')[0]
    prod = prod.strip()
    today = datetime.now(pytz.timezone('America/Mexico_City'))

    header = ['Product', 'Price','Scraping Date']
    data = [title,price,today]

    with open('web data/AmazonWebScraperData.csv','a+',newline='',encoding='utf8') as f:
        writer = csv.writer(f)
        writer.writerow(data)


In [6]:
# This is the main fuction to data over searchs
def DataGathering(obj):  
    try:
        #search and load for obj
        cosa = obj      
        obj = obj.replace(' ','+')
        url = 'https://www.amazon.com.mx/s?k={}&i=pets&rh=p_72%3A7073863011&dc&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=3U4HJF8FHC2PN&qid=1645500646&rnid=7073859011&sprefix=smart+collar%2Cpets%2C176&ref=sr_nr_p_72_1'.format(obj)
        driver.get(url)
        element_present = EC.presence_of_element_located((By.ID, 'search'))
        wait.until(element_present)
        time.sleep(3)
        pag = get_soup(url)
        
        #iteration for price
        spans = pag.find_all("span",attrs={'class':'a-offscreen'})
        cash=[]
        for price in spans:
            price = price.string
            price = price.strip()[1:]
            price = price.split('\n')[0]
            price = price.replace(',','')
            cash.append(price)

        #iteration for prod names
        prods=[]
        spans = pag.find_all("span",attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
        for names in spans:
            names = names.string
            names = names.strip()[0:]
            prods.append(names)

        #iteration for searching index
        search = []
        for _ in itertools.repeat(None, len(prods)):
            search.append(cosa)

        #iteration for getting prod link
        links=[]
        spans = pag.find_all("a",attrs={'class':'a-link-normal s-no-outline'})
        for a in spans:
            #print(a)
            link = a.get('href')
            link = "https://www.amazon.com.mx/"+str(link)
            links.append(link)
        #print status
        status="Prices checked for '{}' in Amazon".format(cosa)

        #start df for saving data.
        cols = ['Search','Product Name','Product Price', 'Link']
        diff = len(prods)-len(cash)
        for _ in itertools.repeat(None, diff):
            cash.append(None)
        df = pd.DataFrame(columns=cols)

        df['Search'] = search
        df['Product Name'] = prods
        df['Product Price'] = cash
        df['Link'] = links

        #clean Nules of price with avg
        mean = df['Product Price'].astype(float).mean()
        #df['Product Price'] = df['Product Price'].astype(float)
        df['Product Price'] = df['Product Price'].replace(np.nan, mean)
        #check previous data
        df_full = pd.read_csv('Data/web data/AmazonWebScraperData.csv')
        #append new data to csv and save
        df_full = df_full.append(df)
        df_full.to_csv('Data/web data/AmazonWebScraperData.csv',index=False)
        print("ok")
    except Exception as e:
        status = "Error. on: " + str(e)
        return status      


In [7]:
# this is to get reviews of each prod
def getReviewsFromProduct(x):
    pag = get_soup(x)
    content = pag.find_all("div", attrs={'data-hook':'review'})
    rev_author=[]
    rev_review=[]
    rev_stars=[]

    for review in content:
        #get name from the review
        try:
            name = review.find('span',attrs={'class':'a-profile-name'})
            name = name.string
            name = name.strip()[0:]
            rev_author.append(name)
        except:
            rev_author.append('unknown')
        #get review
        
        choro = review.find('div', attrs={'data-hook':'review-collapsed'})
        choro = choro.find('span')
        choro = choro.text
        choro = choro.strip()
        choro = re.sub(' +', ' ', choro)
        choro = os.linesep.join([s for s in choro.splitlines() if s])
        rev_review.append(choro)
        #get num of star
        try:
            star = review.find('a', attrs={'class':'a-link-normal'})
            star = star.find('span')
            star = star.text.strip()
            star = star.split(' de')[0]
            rev_stars.append(star)
        except:
            rev_stars.append(0)
        
        #create a df
        cols = ['Review Author','Review Content', 'Stars']
        df = pd.DataFrame(columns=cols)
        df['Review Author']=rev_author
        df['Review Content']=rev_review
        df['Stars']=rev_stars
        #some data cleaning
        df['Review Content'] = df['Review Content'].replace('\n','', regex=True)
        df['Review Content'] = df['Review Content'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        
        #join dfs
        df_full = pd.read_csv('Data/web data/AmazonReviewsData.tsv', sep='|')
        df_full = df_full.append(df)
        df_full.to_csv('Data/web data/AmazonReviewsData.tsv', sep='|',index = False)



## Code
Lets get the best dog collars in the market from our specialists report

In [8]:
pagina = get_soup("https://www.familyhandyman.com/list/best-smart-dog-collars/")
spans = pagina.find_all("h2")
headers=[]
products=[]
for a in spans:
    names = a.string
    names = names.strip()[0:]
    headers.append(names)
for header in headers:
    if ":" in header:
        header = header.split(':')[1]
        header = header[1:]
        products.append(header)
    else:
        None
print(products)

['Fi Smart Dog Collar', 'Whistle FIT', 'Whistle GO', 'Link AKC Smart Collar', 'Pawscout Smarter Pet Tag', 'PitPat 2 Dog Activity Monitor', 'FitBark 2 Dog Activity Monitor']


now that we got the product names from our report lets use our fuctions to gather data of amazon on each product

In [9]:

for product in products:
    DataGathering(product)

ok
ok
ok
ok


In [10]:
df = pd.read_csv('Data/web data/AmazonWebScraperData.csv')
print(df.shape)
df.tail()

(183, 4)


,Search,Product Name,Product Price,Link
178,Pawscout Smarter Pet Tag,Prizi Paw Premio Entrenador y Educador para Co...,450.00,https://www.amazon.com.mx//Prizi-Paw-Entrenado...
179,Pawscout Smarter Pet Tag,Dynotag Etiqueta inteligente para identificaci...,476.09,https://www.amazon.com.mx//Dynotag-Enabled-Dyn...
180,Pawscout Smarter Pet Tag,SynergyLabs Dental Fresh Original Formula for ...,858.00,https://www.amazon.com.mx//SynergyLabs-Dental-...
181,FitBark 2 Dog Activity Monitor,Whistle Fit - Rastreador de Fitness para Masco...,1384.24,https://www.amazon.com.mx//Whistle-GO-Explore-...
182,FitBark 2 Dog Activity Monitor,Umile Nuevo Juguete de Cuerdas para Perro 4 EN...,329.00,https://www.amazon.com.mx//gp/slredirect/picas...


now for each product link we can get the reviews from the page to generate our df for NLP

In [11]:
product_Links = df['Link'].tolist()
i=1
for liga in product_Links:
    print(liga)
    getReviewsFromProduct(liga)
    
    print(i)
    i += 1

https://www.amazon.com.mx//Max-Molly-Comfortable-Protection-Extraordinary/dp/B07S71RTNQ/ref=sr_1_1?__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=3U4HJF8FHC2PN&keywords=Fi+Smart+Dog+Collar&qid=1645669312&refinements=p_72%3A7073863011&rnid=7073859011&s=pet-supplies&sprefix=smart+collar%2Cpets%2C176&sr=1-1
1
https://www.amazon.com.mx//Garmin-Sistema-Seguimiento-Deportivo-Dispositivo/dp/B07GNZ24VD/ref=sr_1_2?__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=3U4HJF8FHC2PN&keywords=Fi+Smart+Dog+Collar&qid=1645669312&refinements=p_72%3A7073863011&rnid=7073859011&s=pet-supplies&sprefix=smart+collar%2Cpets%2C176&sr=1-2&ufe=app_do%3Aamzn1.fos.d6a47de5-06cb-4416-9abc-48deac71c815
2
https://www.amazon.com.mx//Pup-Culture-Airtag-impermeable-rastreadores/dp/B097Z7CYQZ/ref=sr_1_3?__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=3U4HJF8FHC2PN&keywords=Fi+Smart+Dog+Collar&qid=1645669312&refinements=p_72%3A7073863011&rnid=7073859011&s=pet-supplies&sprefix=smart+collar%2Cpets%2C176&sr=1-3
3
https://www.am

## Finished